
# **Using Physicochemical Elements to Predict Wine Quality**

## **Introduction**

 Wine is a complex beverage composed of numerous compounds that contribute to its overall quality and taste. It is viewed as a luxury good and it is enjoyed by many consumers all over the world. The beverage's quality is most commonly assessed through both physicochemical properties and sensory tests (Mor et al.), we used this vital piece of information as the basis for our project. In this data science project, we will determine the most influential physicochemical compounds within wine to most accurately predict overall wine quality.  
 
 <center><img src="data/red and white wine.jpeg"></center>


 Through data analysis, we hope to find predictive relationships in certain compounds that make up wine in hopes of answering our project’s primary question: 
 **How accurately can we predict the quality of wine with the most relevant physicochemical elements using the K-nearest neighbors classification algorithm?**

 We will use a 2009 “Wine Quality” dataset from Portugal which has two dataframes for red wine and white wine. We chose to build our model for predicting ren wine only, as it is the more commonly consumed wine. The data displays several different red wines based on physicochemical tests and their quality from sensory data as a score between 0 and 10. Portugal is one of the top 10 countries that export wine, for which the wine industry is investing in technologies for winemaking and selling. For this dataset, wine certification and quality assessment are key elements. 
 
 These are the columns of the dataset.
 
* 1 - fixed acidity (g(tartaric acid)/dm3) 
* 2 - volatile acidity (g(acetic acid)/dm3)
* 3 - citric acid (g/dm3)
* 4 - residual sugar (g/dm3)
* 5 - chlorides (g(sodium chloride)/dm3)
* 6 - free sulfur dioxide (mg/dm3)
* 7 - total sulfur dioxide (mg/dm3)
* 8 - density (g/cm3)
* 9 - pH
* 10 - sulphates (g(potassium sulphate)/dm3)
* 11 - alcohol (vol.%)

 Output variable (based on sensory data): 
* 12 - quality (score between 0 and 10)


Through comparing relationships between top relevant variables (backed up with research) using visualizations, we will pick the predictors which correlate with quality the most in order to achieve the highest prediction accuracy.


## **Methods and Results**

overall methods

In [1]:
#Loading the relevant libraries

library(tidyverse)
library(tidymodels)
library(repr)
library(RColorBrewer)
library(GGally)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### Loading and Wrangling

In [3]:
wine_data <- read_csv2("https://raw.githubusercontent.com/choialice707/DSCI-100-Group56-Proj/main/winequality-red.csv") |>
    mutate(quality = as_factor(quality)) |>
    filter(alcohol < 150) |>
        mutate(`volatile acidity` = as.numeric(`volatile acidity`),
                `citric acid` = as.numeric(`citric acid`),
                chlorides = as.numeric(chlorides),
                density = as.numeric(density),
                sulphates = as.numeric(sulphates)) |>
na.omit()
head(wine_data)
tail(wine_data)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (5): volatile acidity, citric acid, chlorides, density, sulphates
dbl (2): total sulfur dioxide, quality

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,5
78,0.76,0.04,23,0.092,15,54,0.9970,326,0.65,98,5
112,0.28,0.56,19,0.075,17,60,0.9980,316,0.58,98,6
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
74,0.66,0.00,18,0.075,13,40,0.9978,351,0.56,94,5


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
68,0.620,0.08,19,0.068,28,38,0.99651,342,0.82,95,6
62,0.600,0.08,2,0.090,32,44,0.99490,345,0.58,105,5
59,0.550,0.10,22,0.062,39,51,0.99512,352,0.76,112,6
63,0.510,0.13,23,0.076,29,40,0.99574,342,0.75,11,6
59,0.645,0.12,2,0.075,32,44,0.99547,357,0.71,102,5
6,0.310,0.47,36,0.067,18,42,0.99549,339,0.66,11,6


 The first step in our data analysis, after loading the necessary packgages, was to read the "wine quality" dataset imported onto Jupyter. We then filtered the alcohol variable to be less than 150 to remove any outliers that would otherwise skew the visualization to be clustered in one line. This allowed us to better compare the variables. Finally, we mutated each of the variables with as.numeric to convert the character vectors into a numeric vector, so that the axis numbers are more easily readable.

In [ ]:
wine_data_scaled <- wine_data |> 
 mutate(scaled_fixed_acidity = scale(`fixed acidity`, center = TRUE), 
        scaled_volatile_acidity = scale(`volatile acidity`, center = TRUE),
        scaled_citric_acid = scale(`citric acid`, center = TRUE),
        scaled_chlorides = scale(chlorides, center = TRUE),
        scaled_free_sulfur_dioxide = scale(`free sulfur dioxide`, center = TRUE),
        scaled_total_sulfur_dioxide = scale(`total sulfur dioxide`, center = TRUE),
        scaled_density = scale(density, center = TRUE),
        scaled_pH = scale(pH, center = TRUE),
        scaled_sulphates = scale(sulphates, center = TRUE),
        scaled_alcohol = scale(alcohol, center = TRUE))

head(wine_data_scaled)

 Here the data used was scaled to ensure clear visualizations and facilitate effective comparison of variables since the distances between the points would have otherwise have different significances for the y and x axis . Scaling was performed using standardization, which standardized the range and distribution of the physicochemical elements. This process allowed for a fair and meaningful visual analysis, ensuring that variables with different scales did not dominate the visualizations. As a result, scaling the data improved the clarity and interpretability of the visual comparisons, aiding in the assessment of relationships between the physicochemical elements and wine quality.

### Splitting the Data

In [ ]:
set.seed(1234) #We used the seed to be able to reproduce the results

wine_split <- initial_split(wine_data_scaled, prop = 0.75 , strata = quality)  
wine_train <- training(wine_split)   
wine_test <- testing(wine_split)

head(wine_train)
head(wine_test)

### Preliminary Summary

In [ ]:
wine_summary <- wine_train |>
    group_by(quality) |>
    summarize(count_per_quality = n()) 

wine_summary

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8) 

wine_summary_plot <- wine_summary |>
    ggplot(aes(x = quality, y = count_per_quality)) +
    geom_bar(stat = "identity") +
    labs(x = "Quality of Wine", y = "Total instances")
    
wine_summary_plot

>**Figure 1:** 

Talk ab total wine counts, Talk ab distribution, how it's mostly 5-6 and how this is unbalanced and what we would do, but that it is not working in this case and will discuss further in the discussion.

### Preliminary Visualization for Selecting Predictors

In [ ]:
library(gridExtra)
options(repr.plot.width = 25, repr.plot.height = 6) 

wine_plot1 <- wine_train |>
  ggplot(aes(x = scaled_sulphates, y = scaled_pH, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Sulphates ", 
       y = "pH",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot2 <- wine_train |>
  ggplot(aes(x = scaled_sulphates, y = scaled_total_sulfur_dioxide, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Sulphates ", 
       y = "Total Sulfur Dioxide",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot3 <- wine_train |>
  ggplot(aes(x = scaled_sulphates, y = scaled_alcohol, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Sulphates", 
       y = "Alcohol",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot4 <- wine_train |>
  ggplot(aes(x = scaled_sulphates, y = scaled_volatile_acidity, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Sulphates", 
       y = "Volatile Acidity",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

#############


wine_plot5 <- wine_train |>
  ggplot(aes(x = scaled_pH, y = scaled_total_sulfur_dioxide, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "pH", 
       y = "Total Sulfur Dioxide",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot6 <- wine_train |>
  ggplot(aes(x = scaled_pH, y = scaled_alcohol, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "pH", 
       y = "Alcohol",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot7 <- wine_train |>
  ggplot(aes(x = scaled_pH, y = scaled_volatile_acidity, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "pH", 
       y = "Volatile Acidity",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

#########


wine_plot8 <- wine_train |>
  ggplot(aes(x = scaled_total_sulfur_dioxide, y = scaled_alcohol, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Total_sulfur_dioxide", 
       y = "Scaled Alcohol",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

wine_plot9 <- wine_train |>
  ggplot(aes(x = scaled_total_sulfur_dioxide, y = scaled_volatile_acidity, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Total Sulfur Dioxide", 
       y = "Volatile Acidity",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")

##########


wine_plot10 <- wine_train |>
  ggplot(aes(x = scaled_alcohol, y = scaled_volatile_acidity, color = quality)) +
  geom_point(alpha = 0.6) +
  labs(x = "Alcohol", 
       y = "Volatile Acidity",
       color = "Quality") +
  theme(text = element_text(size = 17))+
scale_color_brewer(palette = "Set2")



grid.arrange(wine_plot1, wine_plot2, wine_plot3, wine_plot4, nrow = 1, top = '.')
grid.arrange(wine_plot5, wine_plot6, wine_plot7, nrow = 1, top = '.')
grid.arrange(wine_plot8, wine_plot9, wine_plot10, nrow = 1, top = '.')

     

>**Figure 2:**

 To determine the variables to include, we referred to a research paper by Cortez et al. as a valuable source of information. According to the findings presented in the paper, the top five relevant variables for predicting wine quality were sulphates, pH, total sulfur dioxide, alcohol, and volatile acidity. We conducted a comparative analysis among these five elements to validate and select the most suitable variables for our prediction model.

 By examining the relationships and patterns between these variables, we aimed to identify which ones exhibited the strongest correlations and clear distributions with wine quality. This was done by visually examining the patterns and trends through plotting each variable against each other. We then assessed the strength of the relationships, selecting the variables with the highest correlations as the most relevant for our model.

 Upon analyzing the results of our comparison, we determined that sulphates, sulfur dioxide, pH values, and alcohol displayed the most consistent and significant associations with wine quality. Therefore, these variables were selected as the most appropriate for inclusion in our prediction model using the KNN classification algorithm.

### Data analysis: Building and Training the Classifier Model

In [ ]:
set.seed(1234)

wine_recipe <- recipe(quality ~ scaled_sulphates + scaled_pH + scaled_total_sulfur_dioxide + scaled_alcohol, data = wine_train)
wine_recipe

wine_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
      set_engine("kknn") |>
      set_mode("classification")
wine_tune


In [ ]:
set.seed(1234)

wine_vfold <- vfold_cv(wine_train, v = 5, strata = quality)

In [ ]:
set.seed(1234)

k_vals <- tibble(neighbors = seq(2, 10, 1))

wine_results <- workflow() |>
      add_recipe(wine_recipe) |>
      add_model(wine_tune) |>
      tune_grid(resamples = wine_vfold, grid = k_vals) |>
      collect_metrics()
wine_results

In [ ]:
set.seed(1234)

wine_accuracies <- wine_results |>
    filter(.metric == "accuracy")
wine_accuracies

In [ ]:
set.seed(1234)

options(repr.plot.width = 7, repr.plot.height = 7) 

accuracy_vs_k_plot <- wine_accuracies |>
    ggplot(aes(x = neighbors, y = mean))+
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") +
    scale_x_continuous(breaks = seq(0, 10, by = 1)) +  # adjusting the x-axis
    scale_y_continuous(limits = c(0.5, 0.6)) # adjusting the y-axis

accuracy_vs_k_plot

>**Figure 3:**

In [ ]:
set.seed(1234)

choice_of_k <- wine_accuracies |>
    arrange(desc(mean)) |>
    slice(1) |>
    pull(neighbors)
choice_of_k

In [ ]:
set.seed(1234) 

wine_recipe_2 <- recipe(quality ~ scaled_sulphates + scaled_pH + scaled_total_sulfur_dioxide + scaled_alcohol, data = wine_train)
wine_recipe_2

wine_spec <-  nearest_neighbor(weight_func = "rectangular", neighbors = choice_of_k)|>
    set_engine("kknn") |>
     set_mode("classification")

wine_spec



In [ ]:
set.seed(1234)

wine_fit <- workflow() |>
add_recipe(wine_recipe_2) |>
add_model(wine_spec) |>
fit(data = wine_train)

wine_fit

In [ ]:
set.seed(1234)

wine_predictions <- predict(wine_fit, wine_test) |>
bind_cols(wine_test)
head(wine_predictions)
wine_metrics <- wine_predictions |>
        metrics(truth = quality, estimate = .pred_class) |> 
filter(.metric == "accuracy")
wine_metrics

wine_conf_mat <- wine_predictions |> 
       conf_mat(truth = quality, estimate = .pred_class)

wine_conf_mat

### Analysis Visualization

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8) 

autoplot(wine_conf_mat, type = "heatmap") +
    scale_fill_distiller(palette = "YlOrRd", name = "Frequency") + 
    labs(title = "Quality Confusion Matrix") +
    theme(legend.position = "left", text = element_text(size = 22),
          plot.caption = element_text(size = 15, hjust = 0))

>**Figure 4:**

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8) 

data <- wine_train %>%
mutate(type = "training") 

data2 <- wine_test %>%
mutate(type = "predictor")
combined <- rbind(data, data2)
head(combined)

ggpairs(combined, aes(colour = type), columns = 1:4)

>**Figure 5:**

## **Discussion**

Include that we tried to balance the data because it was imbalanced (for example there were only 10 of quality 3 and ~600 for 5 and 6) but the themis package wouldnt load. In order  to improve our accuracy we would do the balancing step with the step_upsample function. and also forward selection instead of eyeballing the visualizations.

## **References**

Literature sources:

* Cortez, Paulo, et al. "Modeling wine preferences by data mining from physicochemical properties." Decision Support Systems, ScienceDirect, May 2009, https://www.sciencedirect.com/science/article/abs/pii/S0167923609001377.
 
*  Mor, Nuriel Shalom, et al. "Wine Quality and Type Prediction from Physicochemical Properties Using Neural Networks for Machine Learning: A Free Software for Winemakers and Customers." Journal of Wine Research, Taylor & Francis Online, 2019, https://www.tandfonline.com/doi/full/10.1080/09571264.2019.1590937.

Data source: 

* 